In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim 
from gensim.models.phrases import Phrases, Phraser
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import Counter 
from gensim.models import Word2Vec
from googletrans import Translator
from gensim.models import FastText

In [2]:
data=pd.read_csv("Indian_songs.csv")

In [3]:
data.head()

,movieName,songName,songSinger,songMusic,songLyricist,songLyrics
0,Bala,Don’t Be Shy,"Badshah, Shalmali Kholgade, Gurdeep Mehendi",Sachin-Jigar,"Mellow D, Badshah","Sun, main hoon thoda sanki\nKarun mann ki\nBab..."
1,Laal Kaptaan,Lahu Ka Rang Kara,Samira Koppikar,Samira Koppikar,Sahib,Morey.. lahu ka rang kara\nMorey lahu ka rang ...
2,Laal Kaptaan,Red Red Najariya,Shreya Ghoshal,Samira Koppikar,Saurabh Jain,"Badnaam shehar, badnaam gully\nIsme har raat h..."
3,Laal Kaptaan,Kaal Kaal,"Brijesh Shandilya, Dino James",Samira Koppikar,Saurabh Jain,"Kaal kaal, kaal kaal, jo sapaat chal raha\nWo ..."
4,Laal Kaptaan,Taandav,"Kailash Kher, Brijesh Shandilya",Samira Koppikar,Puneet Sharma,Shor hai andher mein\nJo dher murda pedon ka\n...


In [4]:
doc=[]
for i in range(data.shape[0]):
    doc.append(nltk.word_tokenize(re.sub('[^a-zA-z\s]','',data['songLyrics'][i].lower())))

In [5]:
doc_sent=[]
for i in range(data.shape[0]):
    doc_sent.append(re.sub('[^a-zA-z\s]','',data['songLyrics'][i].lower()).split("\n"))
nonrep_doc=[]
for song in doc_sent:
    d=[]
    for line in song:
        if line not in d:
            d.append(line)
    nonrep_doc.append(d)

In [6]:
nonrep_doc[0]

['sun main hoon thoda sanki',
 'karun mann ki',
 'baby gaana lagade thoda funky',
 'nahi mann ki',
 'nahi dhan ki',
 'ye baat hai tere tann ki',
 '',
 'paagal ho jaaun jab tu ho rubaru',
 'na lamba ocha gora chitta',
 'phir bhi dil mein tu',
 'ishq ne tere kiya hai bekaboo',
 'jaisa hai waisa hi pasand mujhko tu jaanu',
 'i just wanna feel your body',
 'saanu kare ishaare',
 'touch my body',
 'dont be shy my honey',
 'i wanna feel your body',
 'fly my honey',
 'befikar fly my honey',
 'kuch bhi bole bina',
 'aankhon se tu baat kare',
 'ye shaitani jaan bujhke mere saath kare',
 'tujhko dekhe bina dil mera ab kahin lage nahi',
 'tere husn pe aankhein sek sek kar thake nahi',
 'upar se raat hoti ja rahi hai naughty naughty naughty']

In [7]:
doc1=[]
for i in nonrep_doc:
    for j in i:
        if j =='':
            continue
        doc1.append(nltk.word_tokenize(j))

In [8]:
doc1[6]

['paagal', 'ho', 'jaaun', 'jab', 'tu', 'ho', 'rubaru']

In [9]:
def weat_test(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words):
    cos=[]
    s=0
    s1=[]
    s2=[]
    S=[]
    n=0
    for i in range(0, len(target_one_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_one_words[i]
                at1 = attribute_one_words[k]
                try:
                    cos1= model.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_one_words[i]
                at2 = attribute_two_words[k]
                try:
                    cos2= model.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s1.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
            n=n+1
    for i in range(0, len(target_two_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_two_words[i]
                at1 = attribute_one_words[k]
                try:
                    cos1= model.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_two_words[i]
                at2 = attribute_two_words[k]
                try:
                    cos2= model.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s2.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
    s=np.sum(s1)-np.sum(s2)
    stdev=np.std(S)
    print(target_one + ' vs ' + target_two  + ' , ' +attribute_one + ' vs ' + attribute_two +', d = ' + str(s/(stdev*n)))

In [10]:
female_names=['ladki','girl','gori','lady','kudi','chhori','woman']
male_names=['ladka','munda','mundey','boy','chokra','chhora','man']

color =['sanwali','saanwala','pink','pinky','red','laal','kaala','kaali','gori','gora','white','black','yellow','brown']

softAttitude=['bholi','bhola','komalkomal','heeran','hirni','nadaan','beautiful','mastani','mastana','seedhi','seedha','sharmili','sharmeela','sohni','sohna','bhali','bhala']
strongAttitude=['kukkad','bigda','bigdi','khatra','khauf','handa','jungli','badmash','gussa']

cars=['car', 'gaddi','drive','lamborghini','jaguar','gaadi','motorcycle']
clothes=['jeans','skirt','shirt','lehnga','chunni','ainak','ghagra','kurta','pajama','jacket','choodi','jhumka','chasma','chashma','kangan','top']
food=['namkeen','mithi','tikhi','teekha','khatti','makkhan','sweet','nimbu','imli','mitthe','rasmalai','mirchi','mishti','naariyal']
alcohol=['daaru','whisky','daru','pila','botal','peg','shots','drink','peeta']
bodylooks=['choti','chota','cheeks','adayein','thumka','aankhen','aankhein','nazron','charming']

# Word2vec using gensim

In [11]:
model = gensim.models.Word2Vec(doc,size=200,window=15,min_count=2,workers=10,iter=10) 

In [12]:

weat_test('female_names','male_names', female_names, 'softAttitude' ,'strongAttitude', softAttitude, male_names, strongAttitude)

female_names vs male_names , softAttitude vs strongAttitude, d = 0.8058709034607879


/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  from ipykernel import kernelapp as app
/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


# Calculating Average Similarity - word2vec

In [13]:
# li1=['tu','ye','maa','sukun','harami']
# li2=['aaa','gori','laal','sharabi']

In [14]:
def avg_similarity(target,attribute):
   
    S=[]
    
    for i in range(0, len(target)):
        
#         cos.append(model.similarity(target[i],attribute))
#             c1=[]
        maxlist=[]    
        for k in range(0, len(attribute)):
            maxlist.append(model.similarity(target[i], attribute[k]))
            
        maxlist.sort(reverse=True)
        
        for j in range(0,4):
            S.append(maxlist[j])
            
        ans= np.array(S).sum()/(len(target)*4)

        
        
        
    return ans

In [15]:
avg_similarity(female_names,color)

/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()


0.5377033318917798

In [16]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_score=[]
male_score=[]

In [17]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [18]:
for lis in att1:
    
    fem_score.append(avg_similarity(female_names,lis))
    male_score.append(avg_similarity(male_names,lis))

/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()


In [19]:
avg_sim=pd.DataFrame({'attribute':att,'female_names':fem_score,'male_names':male_score})

In [20]:
avg_sim

,attribute,female_names,male_names
0,color,0.537703,0.548279
1,softAttitude,0.485988,0.512739
2,strongAttitude,0.349349,0.495873
3,cars,0.452626,0.505229
4,clothes,0.513804,0.587942
5,food,0.489444,0.563960
6,alcohol,0.386386,0.407034
7,bodylooks,0.458495,0.464155


# FastText

In [21]:
model = FastText(doc, size=100, window=20, min_count=5, workers=4,sg=1)

In [22]:

weat_test('female_names','male_names', female_names, 'softAttitude' ,'strongAttitude', softAttitude, male_names, strongAttitude)

female_names vs male_names , softAttitude vs strongAttitude, d = 1.0143738472471102


/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  from ipykernel import kernelapp as app
/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [23]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_score=[]
male_score=[]

In [24]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [25]:
for lis in att1:
    
    fem_score.append(avg_similarity(female_names,lis))
    male_score.append(avg_similarity(male_names,lis))

/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()


In [26]:
avg_sim=pd.DataFrame({'attribute':att,'female_names':fem_score,'male_names':male_score})

In [27]:
avg_sim

,attribute,female_names,male_names
0,color,0.491935,0.432924
1,softAttitude,0.452269,0.393242
2,strongAttitude,0.397305,0.418502
3,cars,0.423416,0.401536
4,clothes,0.486889,0.479216
5,food,0.444051,0.406869
6,alcohol,0.380756,0.366962
7,bodylooks,0.400594,0.382832


# support and confidence

In [28]:
mctr=0
for d in doc:
    flag=0
    for word in male_names:
        if word in d:
            flag=1
    
    if flag==1:
        mctr=mctr+1
mctr

198

In [29]:
fctr=0
for d in doc:
    flag=0
    for word in female_names:
        if word in d:
            flag=1
    
    if flag==1:
        fctr=fctr+1
fctr

268

In [30]:
total=len(doc)

In [31]:
ctr=0
for d in doc:
    flag=0
    for word in female_names:
        for w2 in alcohol:
            if word in d:
                if w2 in d:
                    flag=1
        
    if flag==1:
        ctr=ctr+1
        
ctr

38

In [32]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_support=[]
male_support=[]
fem_confidence=[]
male_confidence=[]

In [33]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [34]:
for i in att1:
    ctr=0
    for d in doc:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_support.append((ctr/total)*100)

In [35]:
for i in att1:
    ctr=0
    for d in doc:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_support.append((ctr/total)*100)

In [36]:
for i in att1:
    ctr=0
    for d in doc:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_confidence.append((ctr/fctr)*100)

In [37]:
for i in att1:
    ctr=0
    for d in doc:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_confidence.append((ctr/mctr)*100)

In [38]:
support=pd.DataFrame({'attribute':att,'female_names':fem_support,'male_names':male_support})

In [39]:
support

,attribute,female_names,male_names
0,color,2.576809,0.792864
1,softAttitude,1.684836,1.156260
2,strongAttitude,0.297324,0.495540
3,cars,0.759828,0.660720
4,clothes,0.759828,0.495540
5,food,0.330360,0.363396
6,alcohol,1.255368,0.958044
7,bodylooks,1.255368,0.495540


In [40]:
confidence=pd.DataFrame({'attribute':att,'female_names':fem_confidence,'male_names':male_confidence})

In [41]:
confidence

,attribute,female_names,male_names
0,color,29.104478,12.121212
1,softAttitude,19.029851,17.676768
2,strongAttitude,3.358209,7.575758
3,cars,8.582090,10.101010
4,clothes,8.582090,7.575758
5,food,3.731343,5.555556
6,alcohol,14.179104,14.646465
7,bodylooks,14.179104,7.575758


# support confidence on sentences

In [42]:
mctr=0
for d in doc1:
    flag=0
    for word in male_names:
        if word in d:
            flag=1
    
    if flag==1:
        mctr=mctr+1
fctr=0
for d in doc1:
    flag=0
    for word in female_names:
        if word in d:
            flag=1
    
    if flag==1:
        fctr=fctr+1
mctr,fctr

(360, 460)

In [43]:
total=len(doc1)

In [44]:
ctr=0
for d in doc1:
    flag=0
    for word in female_names:
        for w2 in alcohol:
            if word in d:
                if w2 in d:
                    flag=1
        
    if flag==1:
        ctr=ctr+1
        
ctr

3

In [45]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_support=[]
male_support=[]
fem_confidence=[]
male_confidence=[]

In [46]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [47]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_support.append((ctr/total)*100)

In [48]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_support.append((ctr/total)*100)

In [49]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_confidence.append((ctr/fctr)*100)

In [50]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_confidence.append((ctr/mctr)*100)

In [51]:
support1=pd.DataFrame({'attribute':att,'female_names':fem_support,'male_names':male_support})

In [52]:
support1

,attribute,female_names,male_names
0,color,0.081379,0.001356
1,softAttitude,0.021701,0.002713
2,strongAttitude,0.001356,0.009494
3,cars,0.000000,0.000000
4,clothes,0.004069,0.002713
5,food,0.002713,0.000000
6,alcohol,0.004069,0.001356
7,bodylooks,0.001356,0.000000


In [53]:
confidence1=pd.DataFrame({'attribute':att,'female_names':fem_confidence,'male_names':male_confidence})

In [54]:
confidence1

,attribute,female_names,male_names
0,color,13.043478,0.277778
1,softAttitude,3.478261,0.555556
2,strongAttitude,0.217391,1.944444
3,cars,0.000000,0.000000
4,clothes,0.652174,0.555556
5,food,0.434783,0.000000
6,alcohol,0.652174,0.277778
7,bodylooks,0.217391,0.000000


# support confidence paragraph wise

In [55]:
data['songLyrics'][0].split('\n\n')

['Sun, main hoon thoda sanki\nKarun mann ki\nBaby gaana lagade thoda funky\nNahi mann ki\nNahi dhan ki\nYe baat hai tere tann ki',
 'Paagal ho jaaun jab tu ho rubaru\nNa lamba ocha gora chitta\nPhir bhi dil mein tu\nIshq ne tere kiya hai bekaboo\nJaisa hai waisa hi pasand mujhko tu jaanu',
 "I just wanna feel your body\n(Saanu kare ishaare)\nTouch my body\nDon't be shy my honey\n(Saanu kare ishaare)",
 "I wanna feel your body\n(Saanu kare ishaare)\nTouch my body\nDon't be shy my honey\n(Saanu kare ishaare)\nTouch my body\nFly my honey\nDon't be shy my honey\nBefikar fly my honey\nDon't be shy my honey",
 'Sun, main hoon thoda sanki\nKarun mann ki\nBaby gaana lagade thoda funky\nNahi mann ki\nNahi dhan ki\nYe baat hai tere tann ki...',
 "Kuch bhi bole bina\nAankhon se tu baat kare\nYe shaitani jaan bujhke mere saath kare\nTujhko dekhe bina dil mera ab kahin lage nahi\nTere husn pe aankhein sek sek kar thake nahi\nUpar se raat hoti ja rahi hai naughty naughty naughty...\nDon't be shy my 

In [56]:
doc2=[]
for i in range(data.shape[0]):
    doc2.append(re.sub('[^a-zA-z\s]','',data['songLyrics'][i].lower()).split("\n\n"))
doc22=[]
for song in doc2:
    d=[]
    for line in song:
        if line not in d:
            d.append(line)
    doc22.append(d)

In [57]:
doc222=[]
for i in doc22:
    for j in i:
        doc222.append(j.split('\n'))

In [58]:
doc222[0]

['sun main hoon thoda sanki',
 'karun mann ki',
 'baby gaana lagade thoda funky',
 'nahi mann ki',
 'nahi dhan ki',
 'ye baat hai tere tann ki']

In [59]:
doc1=[]
for i in range(len(doc222)):
    doc1.append(nltk.word_tokenize(" ".join(doc222[i])))

In [60]:
mctr=0
for d in doc1:
    flag=0
    for word in male_names:
        if word in d:
            flag=1
    
    if flag==1:
        mctr=mctr+1
fctr=0
for d in doc1:
    flag=0
    for word in female_names:
        if word in d:
            flag=1
    
    if flag==1:
        fctr=fctr+1
mctr,fctr

(224, 307)

In [61]:
total=len(doc1)

In [62]:
ctr=0
for d in doc1:
    flag=0
    for word in female_names:
        for w2 in alcohol:
            if word in d:
                if w2 in d:
                    flag=1
        
    if flag==1:
        ctr=ctr+1
        
ctr

32

In [63]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_support=[]
male_support=[]
fem_confidence=[]
male_confidence=[]


In [64]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [65]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_support.append((ctr/total)*100)

In [66]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_support.append((ctr/total)*100)

In [67]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_confidence.append((ctr/fctr)*100)

In [68]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_confidence.append((ctr/mctr)*100)

In [69]:
support1=pd.DataFrame({'attribute':att,'female_names':fem_support,'male_names':male_support})

In [70]:
support1

,attribute,female_names,male_names
0,color,1.172791,0.344019
1,softAttitude,0.766224,0.469116
2,strongAttitude,0.109461,0.218921
3,cars,0.359656,0.281470
4,clothes,0.359656,0.218921
5,food,0.156372,0.093823
6,alcohol,0.500391,0.375293
7,bodylooks,0.531665,0.187647


In [71]:
confidence1=pd.DataFrame({'attribute':att,'female_names':fem_confidence,'male_names':male_confidence})


In [72]:
confidence1

,attribute,female_names,male_names
0,color,24.429967,9.821429
1,softAttitude,15.960912,13.392857
2,strongAttitude,2.280130,6.250000
3,cars,7.491857,8.035714
4,clothes,7.491857,6.250000
5,food,3.257329,2.678571
6,alcohol,10.423453,10.714286
7,bodylooks,11.074919,5.357143
